In [ ]:
import numpy as np
import pandas as pd
import Levenshtein as lv

from collections import defaultdict as ddict, Counter


# Saud Data

In [ ]:
SD = pd.read_excel("ext.xlsx")
TD = pd.read_csv("sample.tsv", sep="\t")

In [ ]:
SD.columns

In [ ]:
SD.Data_Source_saud_edited.describe()

In [ ]:
SD_IDs = ['BamID_modified_no_dashes','BamID_modified','sample_original','Sauds_BamID_x',
       'sample_id','Sauds_BamID_y','Patient.ID','BamID_modified_discovery','unique_ID']
SD[SD_IDs].describe()

In [ ]:
SD_full_IDs = ['BamID_modified_no_dashes','BamID_modified','sample_original','unique_ID']
SD[SD_full_IDs]

In [ ]:
matches = SD['BamID_modified'] == SD['sample_original']
print(matches.sum())
SD[~matches][['BamID_modified','sample_original']]

# Terra Data

In [ ]:
TD.columns

In [ ]:
TD_IDs = ['entity:sample_id','BamSampleID','edited_participant_id','edited_sample_id','participant_id','Sauds_BamID','participant']
TD[TD_IDs].describe()

In [ ]:
TD_full_IDs=['entity:sample_id','BamSampleID','edited_participant_id', 'participant']
TD[TD_full_IDs]

In [ ]:
2589-2*1341

---
# Compare

In [ ]:
def compare_id_lists(TDIs, SDIs):
    lmn = max([len(x) for x in TDIs])
    rn = max([len(x) for x in SDIs])

    print((" "*lmn) + " | " + " | ".join([f'{x: <{rn}}' for x in SDIs]))

    for x in TDIs:
        print(f'{x: <{lmn}}', end='')
        T_unique = pd.Series(TD[x].unique())
        Tn = len(T_unique)
        for y in SDIs:
            found = T_unique.isin(SD[y]).sum()
            datum = f"{found}/{Tn}"
            print(f' | {datum: >{rn}}', end ='')
        print()


In [ ]:
compare_id_lists(TD_IDs,SD_IDs)

In [ ]:
compare_id_lists(TD_full_IDs, SD_full_IDs)

In [ ]:
T_work = TD[['entity:sample_id','BamSampleID','edited_participant_id', 'participant']].copy()
T_work["Source"] = np.nan
T_work

In [ ]:
S_work = SD[['sample_original','Data_Source_saud_edited']].copy()

In [ ]:
###########################
parcol = ['edited_participant_id', 'participant'][1]
###########################

In [ ]:
missed = []
multi = []
nopid = []
histish = ddict(int)

for _, id, source in S_work.itertuples():
    id_hit = T_work[T_work.BamSampleID == id]
    if len(id_hit) == 0:
        missed.append(id)
        continue

    if len(id_hit) > 1:
        multi.append((id, len(id_hit)))

    n = 0
    for _, sid, bid, pid, s0 in id_hit[['entity:sample_id','BamSampleID', parcol, 'Source']].itertuples():
        if pd.isna(pid):
            nopid.append((sid, bid))
            continue

        if not pd.isna(s0):
            continue

        rows = T_work[parcol] == pid
        n += rows.sum()
        T_work.loc[rows, "Source"] = source

    histish[n] += 1


In [ ]:
print("In Sauds, not matched to Terra BamSampleID:", len(missed))
print("In Sauds, BamSampleID listed multiple times in Terra:", len(multi))
print(len(nopid))
print(histish)

In [ ]:
T_work.describe()

## What just happened?
So, according to the `compare_id_lists(...)` results, 1245 unique BamSampleIDs (of 1446 unique<sup>1</sup>) matched sample_originals (of 1341 unique) from Saud's spreadsheet.
This means 96 of saud's rows (whatever those represent) did not match to any of Terra's _verbatim_.
Further, 131 samples with a matched BamSampleID did not have an edited_participant_ID.

Of the remaining 1114 (=1341-96-131), 2092 samples were able to be traced to the source through fully informed sample_original → BamSampleIDs → edited_participant_id chains.

<sup>1</sup> There is one duplicate BamSampleID (SC_9065 Normal), we'll see what's up with that.

Let's break it down.

In [ ]:
yesBam_noPar = pd.isna(T_work[parcol]) & ~pd.isna(T_work.BamSampleID)
len(T_work[yesBam_noPar])

In [ ]:
noBam_noPar = pd.isna(T_work[parcol]) & pd.isna(T_work.BamSampleID)
len(T_work[noBam_noPar])

Of the 147 Terra samples with no participant ID (which appears to be used to pair Tumor/Normal samples):
 - 134 entries with a BamSampleID don't have a participant ID (131 matched to Saud's data)
 - 13 samples have neither a BamSampleID nor an edited_participant_id

In [ ]:
# break down of the edited_participant_id by multiplicity
par_sets = T_work.groupby(parcol)['entity:sample_id'].count()
Counter(par_sets)

So we have:
- 277 participants with single samples
- 1006 with two samples
- 51 with three

In [ ]:
def do_intersect(n):
    who = par_sets[par_sets == n].index                        # participants with n samples
    qui = T_work[T_work[parcol].isin(who)]       # samples of those participants
    qui2 = qui[~pd.isna(qui.BamSampleID)]                      # ... with BamSampleIDs
    qui3 = qui2[qui2.BamSampleID.isin(S_work.sample_original)] # ... found in Saud's Spreadsheet 

    print(len(qui), len(qui2), len(qui3))
    return qui3

In [ ]:
for i in [1, 2, 3]:
    do_intersect(i)

Of the 277 samples in singles, 204 have BamSampleIDs. 186 BamSampleIDs are in Saud's sample_originals

Of the 2012 samples in the 1006 pairs, 1007 have BamSampleIDs. 878 of those are in Saud's

153 samples in triples, 102 have BamSampleIDs, 50 in Saud's

The numbers suggest in most cases one of a pair has a BamSampleID and 2 of each triple have BamSampleIDs, let's check.
Further, one of each of the triples appears to also be in Saud's, except one case (is it our double?)

Regardless, we have $1114 = 186 + 878 + 50$ and $2092 = 186 + 878*2 + 50*3$, showing that the tool worked as expected, informing participant samples with no BamSampleID of the source via a sample that did.

In [ ]:
who = par_sets[par_sets == 3].index
qui = T_work[T_work[parcol].isin(who)]
bam_break = qui.groupby(parcol)['BamSampleID'].count()
Counter(bam_break)

In [ ]:
who = par_sets[par_sets == 2].index
qui = T_work[T_work[parcol].isin(who)]
bam_break = qui.groupby(parcol)['BamSampleID'].count()
print(Counter(bam_break))
bam_break[bam_break == 2].index

So all of the triples have one sample missing a BamSampleID, and 1005 of the 1006 pairs are split.
There is one pair for which both BamSampleIDs are filled, PRAD-6115392_dot_0.

In [ ]:
who = par_sets[par_sets == 3].index                         # participants with n samples
qui = T_work[T_work[parcol].isin(who)]        # samples of those participants
qui2 = qui[~pd.isna(qui.BamSampleID)]                       # ... with BamSampleIDs
qui3 = qui2[~qui2.BamSampleID.isin(S_work.sample_original)] # ... NOT found in Saud's
qui3[qui3.BamSampleID == 'SC_9065 Normal']

In [ ]:
who = par_sets[par_sets == 2].index                         # participants with n samples
qui = T_work[T_work[parcol].isin(who)]        # samples of those participants
qui2 = qui[~pd.isna(qui.BamSampleID)].copy()                # ... with BamSampleIDs
qui2['In Saud'] = qui2.BamSampleID.isin(S_work.sample_original)
qui2[qui2[parcol] == 'RP-1532_PCProject_0521'] # 'PRAD-6115392_dot_0'

In [ ]:
who = par_sets[par_sets == 3].index
'PRAD-6115392_dot_0' in who

The duplicated BamSampleID _is_ part of a triple, and those BamSampleIDs are _not_ in Saud's spreadsheet.

The one pair for which both BamSampleIDs are present has one of those in Saud's and one not.

# Now What?

### Matched Saud
<table>
  <thead>
    <tr><th colspan=2 scope="row">BamSampleID</th><th colspan="2" scope="col">yes</th><th scope="col">no</th></tr>
    <tr style="border-bottom: double"><th colspan=2 scope="row">sample_original</th><th scope="col">unmatched</th><th scope="col">matched</th><td/><td>96</td></tr>
  </thead>
  <tbody>
    <tr><th rowspan=4 scope="row">edited_participant_id<br>multiplicity</th><th>0</th><td>3</td><td>131</td><td>13</td></tr>
    <tr><th scope="row">1</th><td>18</td><td style="background:#00ff0011">186</td><td>73</td></tr>
    <tr><th scope="row">2</th><td>129</td><td style="background:#00ff0011">878</td><td>1005</td></tr>
    <tr><th scope="row">3</th><td>52</td><td style="background:#00ff0011">50</td><td>51</td></tr>
  </tbody>
</table>

### Source Assigned
<table>
  <thead>
    <tr><th colspan=2 scope="row">BamSampleID</th><th colspan="2" scope="col">yes</th><th scope="col">no</th></tr>
    <tr style="border-bottom: double"><th colspan=2 scope="row">sourced</th><th scope="col">no</th><th scope="col">yes</th><th>yes</th><th>no</th></tr>
  </thead>
  <tbody>
    <tr><th rowspan=4 scope="row">edited_participant_id<br>multiplicity</th><th>0</th><td>134</td><td>0</td><td>0</td><td>13</td></tr>
    <tr><th scope="row">1</th><td>18</td><td style="background:#00ff0011">186</td><td style="background:#00ff0011">0</td><td>73</td></tr>
    <tr><th scope="row">2</th><td>128</td><td style="background:#00ff0011">879</td><td style="background:#00ff0011">877</td><td>128</td></tr>
    <tr><th scope="row">3</th><td>2</td><td style="background:#00ff0011">100</td><td style="background:#00ff0011">50</td><td>1</td></tr>
  </tbody>
</table>

- check the 96 saud samples to see if and where they land in the Terra sample set
- check the 147 samples with no participant id
  - could assign the source for 131 without a listed participant




In [ ]:
def sourced(n):
    who = par_sets[par_sets == n].index                        # participants with n samples
    qui = T_work[T_work[parcol].isin(who)]       # samples of those participants
    qui2 = qui[pd.isna(qui.BamSampleID)]                      # ... with BamSampleIDs
    qui3 = qui2[~pd.isna(qui2.Source)] # ... sourced 

    print(len(qui), len(qui2), len(qui3))
    return qui3

In [ ]:
for i in [1,2,3]:
    sourced(i)

## 96 Saud samples not in Terra

### Levenshtein

In [ ]:
unsourced = T_work[pd.isna(T_work.Source) & ~pd.isna(T_work.BamSampleID)]['BamSampleID']

# Using levenshtein distance to quickly judge similarities
for sam in missed:
    scores = [(lv.distance(sam, x), x) for x in unsourced]
    best = min(scores, key=lambda x:x[0])
    print(sam, best)
        

In [ ]:
# looks like some very bizarre space substitution happened
found = 0
for sam in missed:
    sam_fixed = sam.replace("_space_", " ").strip()
    match_ix = unsourced.str.find(sam_fixed)
    foundit = (match_ix >= 0).sum()
    if foundit > 1: print('bonk', sam, match_ix[match_ix >= 0])
    found += foundit >= 1
found

In [ ]:
T_work2 = T_work.copy()
missed2 = []
multi2 = []
nopid2 = []
histish2 = Counter()

for _, id, source in S_work[S_work.sample_original.isin(missed)].itertuples():
    idf = id.replace("_space_", " ").strip()
    id_hit = T_work2[T_work2.BamSampleID == idf]
    if len(id_hit) == 0:
        missed2.append((id,idf))
        continue

    if len(id_hit) > 1:
        multi2.append((idf, len(id_hit)))

    n = 0
    for _, sid, bid, pid, s0 in id_hit[['entity:sample_id','BamSampleID', parcol, 'Source']].itertuples():
        if pd.isna(pid):
            nopid2.append((sid, bid))
            continue

        if not pd.isna(s0):
            continue

        rows = T_work2[parcol] == pid
        n += rows.sum()
        T_work2.loc[rows, "Source"] = source

    histish2[n] += 1


In [ ]:
T_work2.describe()

In [ ]:
print(histish2)
print(multi2)
# it reports a 6 because SC_9065 gets hit twice each for 3

In [ ]:
for _,m in missed2:
    print(m)

Now we've matched 91 more from Saud's with a little name fixing; 90 pairs and a triple (our double named),
meaning 183 more sourced in the Terra set

### Matched Saud
<table>
  <thead>
    <tr><th colspan=2 scope="row">BamSampleID</th><th colspan="2" scope="col">yes</th><th scope="col">no</th></tr>
    <tr style="border-bottom: double"><th colspan=2 scope="row">sample_original</th><th scope="col">unmatched</th><th scope="col">matched</th><td/><td>5</td></tr>
  </thead>
  <tbody>
    <tr><th rowspan=4 scope="row">edited_participant_id<br>multiplicity</th><th>0</th><td>3</td><td>131</td><td>13</td></tr>
    <tr><th scope="row">1</th><td>18</td><td style="background:#00ff0011">186</td><td>73</td></tr>
    <tr><th scope="row">2</th><td>39</td><td style="background:#00ff0011">968</td><td>1005</td></tr>
    <tr><th scope="row">3</th><td>50</td><td style="background:#00ff0011">52</td><td>51</td></tr>
  </tbody>
</table>

### Source Assigned
<table>
  <thead>
    <tr><th colspan=2 scope="row">BamSampleID</th><th colspan="2" scope="col">yes</th><th scope="col">no</th></tr>
    <tr style="border-bottom: double"><th colspan=2 scope="row">sourced</th><th scope="col">no</th><th scope="col">yes</th><th>yes</th><th>no</th></tr>
  </thead>
  <tbody>
    <tr><th rowspan=4 scope="row">edited_participant_id<br>multiplicity</th><th>0</th><td>134</td><td>0</td><td>0</td><td>13</td></tr>
    <tr><th scope="row">1</th><td>18</td><td style="background:#00ff0011">186</td><td style="background:#00ff0011">0</td><td>73</td></tr>
    <tr><th scope="row">2</th><td>38</td><td style="background:#00ff0011">969</td><td style="background:#00ff0011">967</td><td>38</td></tr>
    <tr><th scope="row">3</th><td>0</td><td style="background:#00ff0011">102</td><td style="background:#00ff0011">51</td><td>0</td></tr>
  </tbody>
</table>



In [ ]:
def do_fixed_intersect(n):
    who = par_sets[par_sets == n].index                                                    # participants with n samples
    qui = T_work2[T_work2[parcol].isin(who)]                                 # samples of those participants
    qui2 = qui[pd.isna(qui.BamSampleID)]                                                  # ... with BamSampleIDs
    
    #qui3 = qui2[qui2.BamSampleID.isin(S_work.sample_original.str.replace("_space_", " ").str.strip())] # ... found in Saud's
    qui3 = qui2[~pd.isna(qui2.Source)]                                                     # ... sourced

    print(len(qui), len(qui2), len(qui3))
    return qui3

In [ ]:
for i in [1, 2, 3]:
    do_fixed_intersect(i)

## 131 Matched BamSampleIDs w/o edited_participant_id

In [ ]:
filter = pd.isna(TD[parcol]) & \
         TD.BamSampleID.isin(SD.sample_original.unique())
bsid = TD[filter].BamSampleID.unique()

filter = pd.isna(T_work2.Source) & \
         ~pd.isna(T_work2[parcol])
unsourced2 = T_work2[filter][parcol].unique()

In [ ]:
for id in bsid:
    scores = [(lv.distance(id, x), x) for x in unsourced2]
    best = min(scores, key=lambda x:x[0])
    print(id, best)

# participant column update

### Matched Saud
<table>
  <thead>
    <tr><th colspan=2 scope="row">BamSampleID</th><th colspan="2" scope="col">yes</th><th scope="col">no</th></tr>
    <tr style="border-bottom: double"><th colspan=2 scope="row">sample_original</th><th scope="col">unmatched</th><th scope="col">matched</th></tr>
  </thead>
  <tbody>
    <tr><th rowspan=4 scope="row">participant<br>multiplicity</th><th>1</th><td>15</td><td style="background:#00ff0011">251</td><td>3</td></tr>
    <tr><th scope="row">2</th><td>39</td><td style="background:#00ff0011">1026</td><td>1063</td></tr>
    <tr><th scope="row">3</th><td>51</td><td style="background:#00ff0011">65</td><td>74</td></tr>
  </tbody>
</table>

### Source Assigned
<table>
  <thead>
    <tr><th colspan=2 scope="row">BamSampleID</th><th colspan="2" scope="col">yes</th><th scope="col">no</th></tr>
    <tr style="border-bottom: double"><th colspan=2 scope="row">sourced</th><th scope="col">no</th><th scope="col">yes</th><th>yes</th><th>no</th></tr>
  </thead>
  <tbody>
    <tr><th rowspan=4 scope="row">edited_participant_id<br>multiplicity</th>
      <th scope="row">1</th><td>15</td><td style="background:#00ff0011">251</td><td style="background:#00ff0011">0</td><td>3</td></tr>
    <tr><th scope="row">2</th><td>38</td><td style="background:#00ff0011">1027</td><td style="background:#00ff0011">1025</td><td>38</td></tr>
    <tr><th scope="row">3</th><td>0</td><td style="background:#00ff0011">116</td><td style="background:#00ff0011">76</td><td>0</td></tr>
  </tbody>
</table>

A lot of the previous discussion is going to look a little non-sensical because I didn't update any of it to reflect the use of the `participant` column over the `edited_participant_id` from the Terra data. That which slips through the gaps. Plus an added fix to the `_space_` fix cleaned up the last five of Saud's not matched to Terra.

So what changed? Well, every sample has a participant listed, not an edited_participant_id, meaning the 147 in the original 0 row got pushed down. Potentially, singles and doubles moved to doubles and triples. This in turn let more be linked and sourced through the BamSampleIDs. It left 94 samples, 76 involved in 38 pairs, unsourced. That's lookable, so let's look.

In [ ]:
T_work2[pd.isna(T_work2.Source)]

# Merging

In [ ]:
T_final = T_work2.set_index('entity:sample_id') \
    .drop(columns='edited_participant_id') \
    .rename(columns={'Source': 'source_study'})
T_final

In [ ]:
source_capture_kit = {
    "AAPC": "Agilent SureSelectv4",
    "Broad/Cornell": "Agilent SureSelectv4",
    "CMI": "Illumina Nextera Rapid Capture",
    "Nelson": "Roche Nimblegen SeqCap v3",
    "NEPC": "Agilent SureSelectv4",
    "PROSNEOLCM": "Agilent SureSelectv4",
    "PCF_SU2C": "Agilent SureSelectv4",
    "TCGA": "Agilent SureSelectv4"
}
with open('justincase' , 'r') as inp:
    datum = inp.readlines()[0].strip()

capture_kit_target_file = {
    "Agilent SureSelectv4": f"gs://{datum}/capture_target_interval_files/SureSelectv4_targets_sorted.bed",
    "Illumina Nextera Rapid Capture": f"gs://{datum}/capture_target_interval_files/NexteraRapidCapture_targets_sorted.bed",
    "Roche Nimblegen SeqCap v3": f"gs://{datum}/capture_target_interval_files/NimblegenSeqCapv3_targets_sorted.bed"
}

source_cap_file = {k : capture_kit_target_file[v] for k,v in source_capture_kit.items()}
source_cap_file = pd.DataFrame(source_cap_file.items(), columns=["source_study", "capture_intervals"]).set_index('source_study')

In [ ]:
TD_inputs = TD[["entity:sample_id", "sample_type", "clean_bam_file_capture", "clean_bai_file_capture"]]\
    .set_index('entity:sample_id') \
    .rename(columns={"clean_bam_file_capture": "clean_bam_capture_hg19", "clean_bai_file_capture": "clean_bai_capture_hg19"})

In [ ]:
Sample_Info = T_final.join(source_cap_file, on="source_study").join(TD_inputs)
Sample_Info.describe()

In [ ]:
Sample_Info.replace(to_replace={
    'clean_bam_capture_hg19' : {'drs://dataguids\.org/': 'drs://dg.4DFC/'},
    'clean_bai_capture_hg19' : {'drs://dataguids\.org/': 'drs://dg.4DFC/'}},
    regex=True, inplace=True)
Sample_Info.loc['PRAD-G9-7522-NB']

# Selection

## no low coverage, post relatedness sample set (1341)

In [ ]:
with open('1341.txt', 'r') as inp, \
     open('1341.list', 'w') as out:
    corpus = inp.read()
    corpus = corpus.replace(', ', '\n')
    out.write(corpus)

In [ ]:
no_low_post_rel = pd.read_csv('1341.list', header=None, names=['entity:sample_id'], index_col=['entity:sample_id'])
Sample_Set = no_low_post_rel.join(Sample_Info)
Sample_Set.describe()

In [ ]:
smol = Sample_Set[~pd.isna(Sample_Set.clean_bam_capture_hg19)] \
    .groupby('source_study') \
    .sample(2)
smol

In [ ]:
smol.to_csv('random_selection.tsv', sep='\t')